# Making Plots from Original images

In [1]:
#### Conda environment
# conda create --name terada2019 python=3.7
# pip install --upgrade pip
# pip install astropy scipy
# pip install photutils
# pip install jupyter matplotlib h5py aplpy pyregion PyAVM healpy

In [2]:
import os
import sys
import time
sys.path
sys.path.append('./')

import numpy as np
from astropy.stats import mad_std

from photutils import datasets
from photutils import DAOStarFinder
from photutils import aperture_photometry, CircularAperture

import aplpy
from astropy.io.fits import getdata
from astropy import wcs
from astropy.io import fits
from astropy import units as u
from astropy import constants as con
from astropy.coordinates import SkyCoord
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
# %matplotlib inline

#### Format of marker file:

name, ra_h ra_m ra_s dec_d dec_m dec_s  R:0-1 G:0-1 B:0-1 alpha  size

dm_tau  04 33 48.7335659850  +18 10 09.974471722  1 0.2 0.2 1.0 2.0

## Defining a aperture photometry pipeline class

In [17]:
class marker:
    '''
    Class for marker objects.
    
    It can also be used as objects to store information about aperture photometry,
    using the attributes :
    

    band_list [list of string] : Recording the information of used bands.
    
    jd_dict   [dictory of (list of double), key: band] : to store jd dates for
                                                         measurements at certain bands.
    
    flux_dict [dictory of (list of double), key: band] : to store fluxes for
                                                         measurements at certain bands
    '''
    
    def __init__(self, label='unknown_marker'):
        self.label = label
        self.ra    = 0.0
        self.dec   = 0.0
        self.color = (0,0,0)
        self.alpha = 1.0
        self.size  = 1.0
        
        self.band_list = []
        self.jd_dict   = {}
        self.flux_dict = {}
    
    def __del__(self):
        pass
        
        

class apt_pipe:
    '''
    Class for apeture photometry pipeline.
    
    History:
    1. Basic version developed.  (Baobab Liu, 2019, Dec.07)
    
    2 Required Conda enviornment:
    # conda create --name terada2019 python=3.7
    # pip install --upgrade pip
    # pip install astropy scipy
    # pip install photutils
    # pip install jupyter matplotlib h5py aplpy pyregion PyAVM healpy
    
    '''

    
    def __init__(self, data_path = './',
                 markerfile   = './markers.txt',
                 ascii_report = './ascii_report.txt',
                 verbose      = False
                ):
        '''
        Initializer of the class.
        It first load image names from the FITS image directory.
        Then it checks the integrity of the FITS image headers,
        and dump report to an ASCII file.
        Finally, it loads the coordinate of the markerfile if any.
        
        
        
        Keywords:
        
        data_path [string] or list of [string] : Default: './'.
                  Can either load data from one specified directory or a list of 
                  specified directory.
        
        markerfile [string]   : File name of one or a list of markers.
                  Format :
                  name, ra_h ra_m ra_s dec_d dec_m dec_s R:0-1 G:0-1 B:0-1 alpha size
                  Example :
                      dm_tau  04 33 48.7335659850 +18 10 09.974471722   1 0.2 0.2 1.0 2.0
                      test    04 33 48.7335659850  +18 10 49.974471722  0.0 1.0 0 1.0 30.0
        
        ascii_report [string] :
        
        verbose   [True]      : Verbosely dump the status of the data.
        
        
        
        Methods:
        
        plot_preview          : Plotting preview figures for FITS images under the data_path(s)
        
        do_apt                : Do aperture photometry for loaded marker(s).
        
        
        '''
                                        
        self.data_path  = data_path
        self.markerfile = markerfile
        self.verbose    = verbose
        self.num_images = 0
        self.num_markers = 0
        self.ascii_report = ascii_report
        
        # initialize variables
        os.system('rm -rf ' + ascii_report)
        F_report = open(ascii_report,"w+")
        self.data_path_list = []
        self.num_data_path  = 0
        self.filterlist     = []
        self.images         = []
        self.path_dict      = {}
        self.band_dict      = {}
        self.date_dict      = {}
        self.jd_dict        = {}
        self.marker_list    = []
        
        # load image names
        if ( type(data_path) == str ):
            self.data_path_list.append(data_path)
            self.num_data_path = 1
            if (verbose==True):
                print('Loading FITS image from single directory : ' + data_path)
                
        if ( type(data_path) == list ):
            self.data_path_list.extend(data_path)
            self.num_data_path = len( self.data_path_list )
            if (verbose==True):
                for data_idx in range(0, len(data_path) ):
                    print('Loading FITS image from : ' + data_path[data_idx])
                            
        try:
            for data_idx in range(0, self.num_data_path):
                self.images.extend(  os.listdir( self.data_path_list[data_idx] )  )
                temp_names = os.listdir( self.data_path_list[data_idx] )
                # self.num_images = self.num_images + len( temp_names )
                for name in temp_names:
                    self.path_dict[name] = self.data_path_list[data_idx]
            self.num_images = len( self.images )
                
            if ( verbose == True ):
                print('##############################################################')
                print('Processing ' + str(self.num_images).strip() + ' images \n')
                print('##############################################################')
        except:
            print('No image found')
            
            
        # checking integrity of FITS headers
        F_report.write('FITS header integrity: \n')
        for i in range(0, self.num_images):
            
            image_name = self.images[i]
            hdulist = fits.open( self.path_dict[image_name] + '/' + image_name)
            try:
                crval1 = hdulist[0].header['crval1']
                crval2 = hdulist[0].header['crval2']
            except:
                if (verbose == True ):
                    print('Warning, coordinate header of ' + image_name + ' does not exist.')
                F_report.write( image_name + ' has no coordinate header \n' )
                continue
                
            try:
                date   = hdulist[0].header['date-obs']
                jd     = hdulist[0].header['jd']
                self.date_dict[image_name] = date
                self.jd_dict[image_name]  = jd
            except:
                if (verbose == True ):
                    print('Warning. Observing date of ' + image_name + ' is not known.')
                F_report.write( image_name + ' has no observing time information \n' )
                continue
                
            try:
                band   = hdulist[0].header['filter']
                if (band not in self.filterlist):
                    self.filterlist.append(band)
                self.band_dict[image_name] = band
            except:
                if (verbose == True ):
                    print('Warning. Filter name of ' + image_name + ' does not exist.')
                F_report.write( image_name + ' has unknown filter band. \n' )
                continue
                
        F_report.write('\n')
        
        # load markers if there is
        try:
            marker_label     = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=0, dtype=np.str)
            rah              = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=1 )
            ram              = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=2 )
            ras              = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=3 )
            decd             = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=4 )
            decm             = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=5 )
            decs             = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=6 )
            markerR          = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=7 )
            markerG          = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=8 )
            markerB          = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=9 )
            marker_alpha     = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=10)
            marker_size      = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=11)
                        
            ra  = ( rah + ram / 60.0 + ras / 3600.0 ) * 15.0
            dec_sign = np.where( (decd>0), 1.0, -1.0)
            dec      = decd + dec_sign * decm / 60.0 + dec_sign * decs / 3600.0
                
            self.num_markers = np.size(ra)
            if (verbose == True):
                print('Number of markers : ' + str(self.num_markers).strip() )
                
            # loading to marker objects
            for mid in range(0, self.num_markers):
                if (self.num_markers > 1):
                    temp_marker       = marker(label=marker_label[mid])
                    temp_marker.ra    = ra[mid] 
                    temp_marker.dec   = dec[mid]
                    temp_marker.color = (markerR[mid], markerG[mid], markerB[mid])
                    temp_marker.alpha = (marker_alpha[mid])
                    temp_marker.size  = (marker_size[mid])
                    self.marker_list.append(temp_marker)
                else:
                    temp_marker       = marker(label=marker_label)
                    temp_marker.ra    = ra
                    temp_marker.dec   = dec
                    temp_marker.color = (markerR, markerG, markerB)
                    temp_marker.alpha = (marker_alpha)
                    temp_marker.size  = (marker_size)
                    self.marker_list.append(temp_marker)
            
        except:
            if (verbose == True):
                print('No markers loaded')

        # Dumping more information ot the report
        # filters
        F_report.write('Used filters: \n')
        outstring = ' '
        for tempstr in self.filterlist:
            outstring = outstring + tempstr + ' '
        F_report.write(outstring + '\n')
        
        F_report.close()

        
        
    def __del__(self):
        pass
    
    
    
    def background_sub(self, image, option='median'):
        '''
        Function to subtract background from a image.
        
        
        Keyword :
        
           option [str]  :
              median : subtracting the median value from an image (Default)
        
        
        Return  :
           Background subtracted image
           
        '''
        
        if ( option == 'median' ):
            image -= np.median(image) 
        
        return image
    
    
    
    def export_apt(self, outfile_name = 'photometry.txt'):
        '''
        Method to output time-sorted photometric measurements to an ascii file.
        For a specific epoch, if a specific band was not observed, then
        the output flux of that band will be 0.0. 
        
        
        Keyword:
        
        outfile_name      [str]  : name of the output file. Default: photometry.txt
        '''

        # Prepare output file
        os.system('rm -rf ' + outfile_name)
        F_out = open(outfile_name,"a+")
        outheader = '# JD     source_name     '
        for band in self.filterlist:
            outheader = outheader + band + '_band_counts     '
        outheader = outheader + '\n'
        F_out.write(outheader)
        
        # collect all possible dates
        temp_jd_list = []
        for marker in self.marker_list:
            for band in marker.band_list:
                temp_jd_list += marker.jd_dict[band]
            # avoid repetition (set) and sort()
            temp_jd_list = list( set(temp_jd_list) )
            temp_jd_list = np.array( temp_jd_list )
            temp_jd_list = np.sort( temp_jd_list )
            
        for jd in temp_jd_list:
            for marker in self.marker_list:
                outstring  = str(jd) + '     ' + str(marker.label) + '     '
                datastring = ''
                temp_flux_dict = {}
                to_output  = False
                
                for band in self.filterlist:
                    temp_flux_dict[band] = 0.0
                
                for band in marker.band_list:
                    try:
                        for record_idx in range( len(marker.jd_dict[band]) ):
                            epoch = marker.jd_dict[band][record_idx]
                            if(epoch==jd):
                                temp_flux_dict[band] = marker.flux_dict[band][record_idx] 
                                to_output = True
                    except:
                        pass
                                
                if ( to_output == True ):
                    for band in self.filterlist:
                        datastring = datastring + str(temp_flux_dict[band]) + '     '
                    outstring = outstring + datastring + '\n'
                    F_out.write(outstring)
        
        F_out.close()
        
        
    
    def do_apt(self, crop_size=1.0/30.0,
                     background_sub_option = 'median',
                     apt_sigmathreshold    = 5.0,
                     fwhm                  = 0.0
              ):
        '''
        Method to do aperture photometry for loaded markers.
        
        
        Keywords : 
        
        crop_size      [float] : size of the crops for making aperture photometry.
                                 Default is 1/30 of the entire image.
                                 
        background_sub_option [str]: Keyword to specify how background subtraction is made.
                                     See description in function: background_sub.
                                     
        apt_sigmathreshold    [float] : Identify stars which are brighter than the specified
                                        sigma threshold. Default: 5-sigma.
                                        
        fwhm                  [float] : fwhm of the point-spread-function in units of pixels.
                                        If not specified, will take values which are obtained from
                                        fitting reference stars.
        
        '''
        F_report = open(self.ascii_report,"a+")
        F_report.write('\n')
        F_report.write('Photometry: \n')
        F_report.write('(only FITS images with compte header information are processed.) \n')
        
        # Obtaining the PSF information (fitting the reference stars if necessary)
        if (fwhm == 0.0):
            if (self.verbose == True):
                print('No input PSF fwhm. Use the fitted values from reference stars')
        
        
        # Doing photometry
        for i in range(0, self.num_images):
            
            image_name = self.images[i]
            hdulist = fits.open(self.path_dict[image_name] + '/' + image_name)
            try:
                image = hdulist[0].data
                naxis1 = hdulist[0].header['naxis1']
                naxis2 = hdulist[0].header['naxis2']                
                w = wcs.WCS(hdulist[0].header)
                cropbox_halfsize_x = int((float(naxis1)/2.0) * crop_size)
                cropbox_halfsize_y = int((float(naxis2)/2.0) * crop_size)
            except:
                F_report.write('Failed to load wcs for image : ' + image_name + '\n')
                continue
                
            try:
                band = self.band_dict[image_name]
                jd   = self.jd_dict[image_name]
            except:
                continue
            
            hdulist.close()
            
            # Process individual markers
            for marker in self.marker_list:
                
                # load pixel coordinates
                x, y = marker.ra, marker.dec
                pixcrd2 = w.wcs_world2pix([ [x,y] ], 0)
                xpix = pixcrd2[0][0]
                ypix = pixcrd2[0][1]
                xpix_min = int(round(xpix - cropbox_halfsize_x) )
                xpix_max = int(round(xpix + cropbox_halfsize_x) )
                ypix_min = int(round(ypix - cropbox_halfsize_y) )
                ypix_max = int(round(ypix + cropbox_halfsize_y) )
                
                if (xpix_min < 0): xpix_min = 0
                if (ypix_min < 0): ypix_min = 0
                if (xpix_max > (naxis1-1) ): xpix_min = (naxis1-1)
                if (ypix_max > (naxis2-1) ): ypix_min = (naxis2-1)
                
                # crop the image to speed up the source finding
                crop = image[ypix_min:ypix_max, xpix_min:xpix_max].astype(float)
                
                # do background subtraction from the crop
                crop = self.background_sub(crop, option=background_sub_option)
                
                # estimate robust standard deviation from the cropped image
                bkg_sigma = mad_std(crop)
                
                # find stars
                daofind = DAOStarFinder(fwhm=fwhm, threshold=apt_sigmathreshold*bkg_sigma)
                sources = daofind(crop)
                try:
                    mxcentroid_array = np.array( sources['xcentroid'] )
                    mycentroid_array = np.array( sources['ycentroid'] )
                    mpeak_array      = np.array( sources['peak']      )
                    num_sources      = len(mpeak_array)
                except:
                    num_sources      = 0
                
                # Outputting measurements
                if ( num_sources > 0 ):
                    sortidx    = np.argsort(mpeak_array)
                    maxidx     = sortidx[-1]
                    mxcentroid = mxcentroid_array[maxidx]
                    mycentroid = mycentroid_array[maxidx]
                    positions = np.transpose((mxcentroid, mycentroid))
                    apertures = CircularAperture(positions, fwhm*2.0)
                    phot_table = aperture_photometry(crop, apertures)
                    counts    = phot_table['aperture_sum'][0]
                    
                    if (band not in marker.band_list):
                        marker.band_list.append( band )
                        # initializing lists to store JD and counts
                        marker.jd_dict[ band ]     = []
                        marker.flux_dict[ band ]   = []
                        
                    marker.jd_dict[ band ].append( jd )
                    marker.flux_dict[ band ].append( counts )


                else:
                    F_report.write("No source found in" + image_name)
           
        F_report.write('\n')
        F_report.close()
    
    
    
    def plot_preview(self, output_directory='./preview_images',
                     label_marker=True,
                     fig_format='png'
                    ):
        '''
        Method to plot the preview figures for FITS imagfes.
        It will produce figures for the observed images (one sub-directory for each band),
        and collect them into an output_directory.
        
        
        Keywords:
        
        output_directory [string] : The directory to collect the output images.
        
        label_marker [True/False] : If True, label the names of the markers on the figure.
        
        fig_format   [string]     : 'png' or 'pdf'
        '''
        
        F_report = open(self.ascii_report,"a+")
        F_report.write('\n')
        F_report.write('Image preview summary: \n')
        
        os.system('rm -rf ' + output_directory)
        os.system('mkdir '  + output_directory)
        
        output_path_dict = {}
        for tempstr in self.filterlist:
            output_path_dict[tempstr] = './' + tempstr + 'band_maps'
            
        for key in output_path_dict.keys():
            os.system('rm -rf ' + output_path_dict[key] )
            os.system('mkdir '  + output_path_dict[key] )
            
        # plot images
        for i in range(0, self.num_images):
            image_name = self.images[i]
            try:
                band       = self.band_dict[image_name]
                jd         = self.jd_dict[image_name]
            except:
                F_report.write('Warning. Image ' +  image_name + ' is not saved. \n')
                continue
            
            # plot image
            fig = aplpy.FITSFigure( self.path_dict[image_name] + '/' + image_name)
            fig.show_grayscale(invert=False)
            
            # plot symbol
            
            # define output figure name
            try:
                outfig_name = \
                              band + '_' + \
                              str( round(jd,5) ) + '.' + fig_format
                # fig.set_xaxis_coord_type('longitude')
                # fig.set_yaxis_coord_type('latitude')
                fig.axis_labels.hide()
                fig.show_grayscale(invert=False)
            except:
                if (self.verbose == True):
                    F_report.write('Warning. Image ' +  image_name + ' is not saved. \n')
                    
            # plot markers
            
            for marker in self.marker_list:
                # plot markers in the png figure
                fig.show_markers(
                                 marker.ra, marker.dec, 
                                 edgecolor=marker.color, 
                                 # facecolor=facecolor[plot_id],
                                 marker='o', s=marker.size, 
                                 alpha=marker.alpha
                                )
                
                if (label_marker == True):
                    label = marker.label
                    fig.add_label(marker.ra, marker.dec, '  ' + label,
                                  color=marker.color, fontsize=12, horizontalalignment='left')
                
            # label date
            try:
                date = self.date_dict[image_name].strip()
            except:
                date = 'unknown_date'
                
            date_label = 'Date : ' + date + '  JD : ' \
                         +  str(jd).strip()                    \
                         + '   Band : ' + band  
            fig.add_label(0.02, 0.95, date_label, relative=True, 
                         color=(0,1,1,1),
                         fontsize=9, horizontalalignment='left')
                    
            fig.axis_labels.hide()
            fig.save(outfig_name)    
            os.system('mv ' + outfig_name + ' ' + output_path_dict[band] )
            
        # pack output maps to be under the same directory
        for key in output_path_dict.keys():
            os.system('mv ' + output_path_dict[key] + ' ./' +  output_directory)   

        F_report.close()

In [18]:
'''
dmtau = apt_pipe(data_path = r"../3486034-dm tau",
                 markerfile   = './markers.txt', verbose=True)
dmtau.plot_preview(label_marker=False)
'''

'\ndmtau = apt_pipe(data_path = r"../3486034-dm tau",\n                 markerfile   = \'./markers.txt\', verbose=True)\ndmtau.plot_preview(label_marker=False)\n'

In [20]:
dmtau = apt_pipe(data_path = [
                              r"../3463766-dm tau",
                              r"../3486034-dm tau",
                              r"../3488473-dm tau",
                             ],
                 markerfile   = './markers.txt', verbose=True)

# dmtau.plot_preview(label_marker=False)

dmtau.do_apt(fwhm=5.0)
dmtau.export_apt()

#for marker in dmtau.marker_list:
#    print(marker.label, ':', marker.band_list)
#    for band in marker.band_list:
#        print('Band : ', band)
#        print(marker.jd_dict[band])
#        print(marker.flux_dict[band])
#        print('\n')

Loading FITS image from : ../3463766-dm tau
Loading FITS image from : ../3486034-dm tau
Loading FITS image from : ../3488473-dm tau
##############################################################
Processing 117 images 

##############################################################
Warning, coordinate header of dm tau_3463766_V_002.fits does not exist.
Warning, coordinate header of dm tau_3463766_I_008.fits does not exist.
Warning, coordinate header of dm tau_3463766_V_000.fits does not exist.
Warning, coordinate header of dm tau_3463766_R_004.fits does not exist.
Warning, coordinate header of dm tau_3463766_R_005.fits does not exist.
Warning, coordinate header of dm tau_3463766_R_003.fits does not exist.
Warning, coordinate header of dm tau_3463766_I_006.fits does not exist.
Warning, coordinate header of dm tau_3463766_I_007.fits does not exist.
Warning, coordinate header of dm tau_3463766_V_001.fits does not exist.
Warning, coordinate header of dm tau_3488473_I_047.fits does not exist

In [ ]:
'''
# Directory for input image
data_path   = r"../3488473-dm tau"

# Defining output directories
output_path_dict = {}
output_path_dict['R'] = './R_maps'
output_path_dict['V'] = './V_maps'
output_path_dict['I'] = './I_maps'

# set if we want to do photometry for objects in the markers list
dophotometry     = True
aperature_r_pix  = 10.0 # pixels
if (dophotometry == True):
    print("Will output photometry. Please define the output filename if not yet.")
    photometry_file_dict = {}
    photometry_file_dict['R'] = './R_phot.txt'
    photometry_file_dict['V'] = './V_phot.txt'
    photometry_file_dict['I'] = './I_phot.txt'
    

# Output ASCII list for FITS images which do not possess coordinate headers
f = open("NO_COORD_HEADER_list.txt","w")    
    
markerfile = './markers.txt'
    
# Preparation ##############################################################
for key in output_path_dict.keys():
    os.system('rm -rf ' + output_path_dict[key] )
    os.system('mkdir '  + output_path_dict[key] )
    
if (dophotometry == True):
    for key in photometry_file_dict.keys():
        os.system('rm -rf ' + photometry_file_dict[key] )
        photo_file = open(photometry_file_dict[key],"a+")
        photo_file.write("# Data  target_name  JD  Counts")

num_markers = 0
try:
    marker_label = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=0, dtype=np.str)
    rah, ram, ras = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(1,2,3) )
    decd, decm, decs = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(4,5,6) )
    markerR, markerG, markerB = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(7,8,9) )
    marker_alpha = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=10)
    marker_size  = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=11)
    
    ra  = ( rah + ram / 60.0 + ras / 3600.0 ) * 15.0
    if (decd > 0 ):
        dec = decd + decm / 60.0 + decs / 3600.0
    else:
        dec = decd - decm / 60.0 - decs / 3600.0
    num_markers = np.size(ra)
    
except:
    print('No markers found')
    
images     = os.listdir( data_path )
num_images = len(images)
############################################################################
    

for i in range(0, num_images):
    image_name = images[i]
    
    info = image_name.strip('.fits').split('_')
    target_name    = info[0]
    directory_name = info[1]
    band           = info[2]
    epoch_idx      = info[3]
    
    # open FITS image
    hdulist = fits.open(data_path + '/' + image_name)
    try:
        crval1 = hdulist[0].header['crval1']
        crval2 = hdulist[0].header['crval2']
        date   = hdulist[0].header['date-obs']
        jd     = hdulist[0].header['jd']
    except:
        print('Error, coordinate header of ' + image_name + ' does not exist.')
        f.write( image_name + '\n' )
        continue

    # loading image
    if (dophotometry == True):
        try:
            image = hdulist[0].data
            w = wcs.WCS(hdulist[0].header)
        except:
            print("Error loading image " + directory_name + '_' + band + '_' + epoch_idx)
        
    
    fig = aplpy.FITSFigure(data_path + '/' + image_name)
    #fig.set_xaxis_coord_type('longitude')
    #fig.set_yaxis_coord_type('latitude')
    fig.axis_labels.hide()
    fig.show_grayscale(invert=False)
    
    # mark stars
    for j in range(0, num_markers):
        # plot markers in the png figure
        if (num_markers==1):
            x, y = ra, dec
            pixcrd2 = w.wcs_world2pix([ [x,y] ], 0)
            xpix = pixcrd2[0][0]
            ypix = pixcrd2[0][1]
            mcolor = (markerR, markerG, markerB)
            malpha = marker_alpha
            msize  = marker_size
            mlabel = str(marker_label)
        else:
            x, y = ra[j], dec[j]
            mcolor = (markerR[j], markerG[j], markerB[j])
            malpha = marker_alpha[j]
            msize = marker_size[j]
            mlabel = str(marker_label[j])
        
        fig.show_markers(
                         x, y, 
                         edgecolor=mcolor, 
                         # facecolor=facecolor[plot_id],
                         marker='o', s=msize, 
                         alpha=malpha
                        )
        fig.add_label(x, y, '  ' + mlabel, 
                      color=mcolor, fontsize=12, horizontalalignment='left')
        
        # optionally, do aperture photometry
        if (dophotometry == True):              
            # making photometry
            xpix_min = int(round(xpix - aperature_r_pix*3) )
            xpix_max = int(round(xpix + aperature_r_pix*3) )
            ypix_min = int(round(ypix - aperature_r_pix*3) )
            ypix_max = int(round(ypix + aperature_r_pix*3) )
            try:
                print(directory_name + '_' + band + '_' + epoch_idx)
                crop = image[ypix_min:ypix_max, xpix_min:xpix_max].astype(float)
                
                # background subtraction
                crop -= np.median(crop)
                
                # estimate pixel statistics
                bkg_sigma = mad_std(crop)
                
                # find stars
                daofind = DAOStarFinder(fwhm= aperature_r_pix/2.0, threshold=5.*bkg_sigma)
                sources = daofind(crop)

                
                mxcentroid_array = np.array( sources['xcentroid'] )
                mycentroid_array = np.array( sources['ycentroid'] )
                mpeak_array      = np.array( sources['peak']      )
                
                if ( len(mpeak_array) > 0 ):
                    # Very ugly code here due to unfamiliar with python.
                    # need to update. Baobab, 2019.Dec.25
                    index_array = range(0, len(mpeak_array) )
                    xindex = np.max(np.where( mpeak_array == np.max(mpeak_array), index_array, -1))
                    yindex = np.max(np.where( mpeak_array == np.max(mpeak_array), index_array, -1))
                    mxcentroid = mxcentroid_array[xindex]
                    mycentroid = mycentroid_array[yindex]
                    positions = np.transpose((mxcentroid, mycentroid))
                    apertures = CircularAperture(positions, aperature_r_pix)
                    phot_table = aperture_photometry(crop, apertures)
                    counts    = phot_table['aperture_sum'][0]

                    try:
                        photo_file = open(photometry_file_dict[band],"a+")
                        outtext     = directory_name + '_' + band + '_' + epoch_idx + '  ' + \
                                      str(marker_label) + '   ' + \
                                      str(jd).strip() + '   ' + \
                                      str(counts) + '   \n'
                        photo_file.write(outtext)
                        photo_file.close()
                    except:
                        print('Error opening output file. ' + directory_name + '_' + band + '_' + epoch_idx)
                    
                else:
                    print("No source found in" + directory_name + '_' + band + '_' + epoch_idx)
                    
                
            except:
                print("Error making photometry " + directory_name + '_' + band + '_' + epoch_idx)
                
            
        
    # label date
    date_label = 'Date : ' + date + '  JD : ' + str(jd).strip() + '   Band : ' + band  
    fig.add_label(0.02, 0.95, date_label, relative=True, 
                  color=(0,1,1,1),
                  fontsize=9, horizontalalignment='left')


    
    outfig_name = directory_name + '_' + band + '_' + epoch_idx + '.png'
    
    fig.save(outfig_name)    
    os.system('mv ' + outfig_name + ' ' + output_path_dict[band] )
    
    # close FITS image
    hdulist.close()
    
f.close()
'''

In [ ]:
test = ''
print(len(test))